In [3]:
!pip install yfinance


In [4]:
pip install fredapi


Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gargi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
pip install textblob nltk


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install lightgbm


Note: you may need to restart the kernel to use updated packages.


Fetching data for TSLA...
Fetching data for AMD...
Fetching data for SQ...
Fetching data for ROKU...
Fetching data for ZM...
Fetching data for PTON...
Fetching data for SPCE...
Fetching data for MRNA...
Fetching data for PLTR...
Fetching data for PINS...
Fetching data for BYND...
Fetching data for DOCU...
Fetching data for COIN...
Fetching data for CRWD...
Fetching data for NIO...
Fetching data for ETSY...
Fetching data for TWLO...
Fetching data for W...
Fetching data for ZG...
Fetching data for FSLY...
Fetching data for SHOP...
Fetching data for BIDU...
Fetching data for BABA...
Fetching data for SPOT...
Fetching data for U...
Fetching data for TTD...
Fetching data for OKTA...
Fetching data for TEAM...
Fetching data for DOCU...
Fetching data for NET...


HEXO: No timezone found, symbol may be delisted
APHA: No timezone found, symbol may be delisted
CLDR: No timezone found, symbol may be delisted


Fetching data for TDOC...
Fetching data for DDOG...
Fetching data for SPCE...
Fetching data for NKLA...
Fetching data for CGC...
Fetching data for HEXO...
Fetching data for ACB...
Fetching data for CRON...
Fetching data for TLRY...
Fetching data for APHA...
Fetching data for SNDL...
Fetching data for IRBT...
Fetching data for CLDR...
Fetching data for SNOW...
Fetching data for ASAN...
Fetching data for LMND...
Fetching data for VEEV...
Fetching data for MDB...
Fetching data for FIL-USD...
Fetching data for UNI7083-USD...
Epoch 1/100


c:\Users\gargi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


426/426 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1251.6829 - val_loss: 348.3125
Epoch 2/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 99.1446 - val_loss: 278.3181
Epoch 3/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 90.3724 - val_loss: 469.8478
Epoch 4/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 78.2124 - val_loss: 433.3322
Epoch 5/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 74.8309 - val_loss: 377.8325
Epoch 6/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 77.1140 - val_loss: 554.5185
Epoch 7/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 73.9349 - val_loss: 326.3806
Epoch 8/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 70.4357 - val_loss: 268.1584
Epoch 9/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 63.5983 - val_loss: 372.5905
Epoch 10/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 66.2289 - val_loss: 478.1296
Epoch 11/100
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 60.8486 - val_loss: 368.9952
Epoch 12/100
426/4

NameError: name 'days' is not defined

In [ ]:
import pandas as pd
import sqlite3

# Load Excel data
df = pd.read_excel('final_stock_data.xlsx')

# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')

# Save DataFrame to SQLite
df.to_sql('stock_data', conn, if_exists='replace', index=False)

# Close connection
conn.close()


In [8]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def fetch_and_save_data(stocks):
    all_data = []

    for symbol in stocks:
        print(f"Fetching data for {symbol}...")
        end_date = datetime.date.today()
        start_date = end_date - datetime.timedelta(days=730)
        stock = yf.Ticker(symbol)
        
        stock_data = stock.history(start=start_date, end=end_date)
        if not stock_data.empty:
            stock_data.reset_index(inplace=True)
            stock_data['Symbol'] = symbol
            stock_data['Dividends'] = stock.dividends.reindex(stock_data['Date'], fill_value=0)
            stock_data['Stock Splits'] = stock.splits.reindex(stock_data['Date'], fill_value=0)
            stock_data['Date'] = stock_data['Date'].dt.tz_localize(None)

            stock_data['Target_3d'] = stock_data['Close'].shift(-3)
            stock_data['Target_10d'] = stock_data['Close'].shift(-10)
            stock_data['Sequence'] = stock_data.groupby('Symbol').cumcount() + 1

            all_data.append(stock_data)

    df = pd.concat(all_data, ignore_index=True)

    # Feature engineering
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek

    def compute_rsi(series, window=14):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    df['SMA'] = df.groupby('Symbol')['Close'].rolling(window=14).mean().reset_index(level=0, drop=True)
    df['RSI'] = df.groupby('Symbol')['Close'].apply(lambda x: compute_rsi(x)).reset_index(level=0, drop=True)
    exp1 = df.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
    exp2 = df.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
    df['MACD'] = exp1 - exp2
    df['BB_Middle'] = df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=20).mean())
    df['BB_Upper'] = df['BB_Middle'] + 2 * df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=20).std())
    df['BB_Lower'] = df['BB_Middle'] - 2 * df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=20).std())

    # Handle missing values appropriately
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df.fillna(0, inplace=True)

    # Normalize the features
    scaler = MinMaxScaler()
    numeric_columns = df.select_dtypes(include=[np.number]).columns.difference(['Close', 'Target_3d', 'Target_10d', 'Stock Splits'])
    df_scaled = df.copy()
    df_scaled[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    # Select only the required columns
    required_columns = [
        'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 
        'Symbol', 'Target_3d', 'Target_10d', 'Sequence', 'Year', 'Month', 'Day', 'DayOfWeek', 
        'SMA', 'RSI', 'MACD', 'BB_Middle', 'BB_Upper', 'BB_Lower'
    ]
    df_scaled = df_scaled[required_columns]
    
    # Save the feature names
    joblib.dump(numeric_columns, 'feature_names.pkl')
    # Save the final cleaned and scaled data
    df_scaled.to_excel('final_stock_data.xlsx', index=False)

    return df, df_scaled, numeric_columns, scaler

# Initialize stocks and fetch data
stocks = ["TSLA", "AMD", "SQ", "ROKU", "ZM", "PTON", "SPCE", "MRNA", "PLTR", "PINS", 
          "BYND", "DOCU", "COIN", "CRWD", "NIO", "ETSY", "TWLO", "W", "ZG", "FSLY", 
          "SHOP", "BIDU", "BABA", "SPOT", "U", "TTD", "OKTA", "TEAM", "DOCU", "NET", 
          "TDOC", "DDOG", "SPCE", "NKLA", "CGC", "HEXO", "ACB", "CRON", "TLRY", "APHA", 
          "SNDL", "IRBT", "CLDR", "SNOW", "ASAN", "LMND", "VEEV", "MDB","FIL-USD", "UNI7083-USD"]

df, df_scaled, numeric_columns, scaler = fetch_and_save_data(stocks)

# Create inputs for CNN (ticker-wise)
tickers = df_scaled['Symbol'].unique()
X, y_3d, y_10d = [], [], []

sequence_length = 60  # Define the sequence length for the CNN

for ticker in tickers:
    df_ticker = df_scaled[df_scaled['Symbol'] == ticker].sort_values('Date')
    ticker_data = df_ticker[numeric_columns].values
    
    # Ensure that the target values are referenced correctly and safely
    if len(df_ticker) > sequence_length + 10:
        for i in range(len(df_ticker) - sequence_length - 10):
            X.append(ticker_data[i:i+sequence_length])
            y_3d.append(df_ticker['Target_3d'].values[i+sequence_length])
            y_10d.append(df_ticker['Target_10d'].values[i+sequence_length])

X = np.array(X)
y_3d = np.array(y_3d)
y_10d = np.array(y_10d)

# Ensure correct shape for model training
X = X.reshape(-1, sequence_length, X.shape[-1])
y_3d = y_3d.reshape(-1, 1)
y_10d = y_10d.reshape(-1, 1)

# Split data into train and test sets (e.g., 80-20 split)
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train_3d, y_test_3d = y_3d[:split_index], y_3d[split_index:]
y_train_10d, y_test_10d = y_10d[:split_index], y_10d[split_index:]

def create_cnn_model(input_shape, kernel_size):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model


# Create and train models for both 3-day and 10-day predictions
model_cnn_3d = create_cnn_model((X_train.shape[1], X_train.shape[2]))
model_cnn_10d = create_cnn_model((X_train.shape[1], X_train.shape[2]))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train 3-day model
history_3d = model_cnn_3d.fit(X_train, y_train_3d, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Train 10-day model
history_10d = model_cnn_10d.fit(X_train, y_train_10d, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate models
preds_cnn_3d = model_cnn_3d.predict(X_test).flatten()
mse_3d = mean_squared_error(y_test_3d, preds_cnn_3d)
mae_3d = mean_absolute_error(y_test_3d, preds_cnn_3d)
r2_3d = r2_score(y_test_3d, preds_cnn_3d)

preds_cnn_10d = model_cnn_10d.predict(X_test).flatten()
mse_10d = mean_squared_error(y_test_10d, preds_cnn_10d)
mae_10d = mean_absolute_error(y_test_10d, preds_cnn_10d)
r2_10d = r2_score(y_test_10d, preds_cnn_10d)

print(f'3-Day Model - MSE: {mse_3d}, MAE: {mae_3d}, R²: {r2_3d}')
print(f'10-Day Model - MSE: {mse_10d}, MAE: {mae_10d}, R²: {r2_10d}')

# Predict future prices
def generate_future_dates(start_date, days):
    return [start_date + datetime.timedelta(days=i) for i in range(1, days + 1)]

def predict_future_prices_cnn(ticker, df, df_scaled, model_cnn_3d, model_cnn_10d, scaler, sequence_length=60, days=[3, 10]):
    df_ticker = df[df['Symbol'] == ticker]
    df_ticker_scaled = df_scaled[df_scaled['Symbol'] == ticker]
    
    X = df_ticker_scaled[numeric_columns].values
    last_sequence = X[-sequence_length:]
    
    future_dates = generate_future_dates(df_ticker['Date'].max(), max(days))
    predictions = {day: None for day in days}

    prediction_3d = None
    prediction_10d = None

    for i in range(max(days)):
        prediction_input = last_sequence.reshape(1, sequence_length, len(numeric_columns))

        if i + 1 == 3:
            prediction_3d = model_cnn_3d.predict(prediction_input)[0, 0]
            predictions[3] = prediction_3d
        if i + 1 == 10:
            prediction_10d = model_cnn_10d.predict(prediction_input)[0, 0]
            predictions[10] = prediction_10d

        # Update last sequence with the prediction
        last_sequence = np.roll(last_sequence, -1, axis=0)
        if prediction_3d is not None and i < 3:
            last_sequence[-1, 0] = prediction_3d
        if prediction_10d is not None and i < 10:
            last_sequence[-1, 0] = prediction_10d

    # Inverse transform the predictions
    for day in days:
        if predictions[day] is not None:
            predictions[day] = scaler.inverse_transform(
                np.concatenate((np.array(predictions[day]).reshape(1, 1), 
                                np.zeros((1, len(numeric_columns) - 1))), axis=1)
            )[0, 0]

    return predictions

# Predict for each ticker
ticker_predictions = {}
days = [3, 10]
for ticker in df['Symbol'].unique():
    predictions = predict_future_prices_cnn(ticker, df, df_scaled, model_cnn_3d, model_cnn_10d, scaler, sequence_length=60, days=days)
    
    df_ticker = df[df['Symbol'] == ticker]
    future_dates = generate_future_dates(df_ticker['Date'].max(), max(days))

    plt.figure(figsize=(14, 5))
    plt.plot(df_ticker['Date'], df_ticker['Close'], label='Actual Prices')
    for day in days:
        plt.scatter(future_dates[day-1], predictions[day], color='red', zorder=5, label=f'{day}-Day Prediction: {predictions[day]:.2f}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Future Stock Price Prediction for {ticker}')
    plt.legend()
    plt.show()


Fetching data for TSLA...
Fetching data for AMD...
Fetching data for SQ...
Fetching data for ROKU...
Fetching data for ZM...
Fetching data for PTON...
Fetching data for SPCE...
Fetching data for MRNA...
Fetching data for PLTR...
Fetching data for PINS...
Fetching data for BYND...
Fetching data for DOCU...
Fetching data for COIN...
Fetching data for CRWD...
Fetching data for NIO...
Fetching data for ETSY...
Fetching data for TWLO...
Fetching data for W...
Fetching data for ZG...
Fetching data for FSLY...
Fetching data for SHOP...
Fetching data for BIDU...
Fetching data for BABA...
Fetching data for SPOT...
Fetching data for U...
Fetching data for TTD...
Fetching data for OKTA...
Fetching data for TEAM...
Fetching data for DOCU...
Fetching data for NET...
Fetching data for TDOC...
Fetching data for DDOG...


HEXO: No timezone found, symbol may be delisted
APHA: No timezone found, symbol may be delisted
CLDR: No timezone found, symbol may be delisted


Fetching data for SPCE...
Fetching data for NKLA...
Fetching data for CGC...
Fetching data for HEXO...
Fetching data for ACB...
Fetching data for CRON...
Fetching data for TLRY...
Fetching data for APHA...
Fetching data for SNDL...
Fetching data for IRBT...
Fetching data for CLDR...
Fetching data for SNOW...
Fetching data for ASAN...
Fetching data for LMND...
Fetching data for VEEV...
Fetching data for MDB...
Fetching data for FIL-USD...
Fetching data for UNI7083-USD...


TypeError: create_cnn_model() missing 1 required positional argument: 'kernel_size'